# 🏔️ Entrenamiento Interactivo de Modelos Sísmicos
## Sistema de IA para Monitoreo de Deformación Sísmica

Este notebook permite entrenar modelos de machine learning para la predicción de deformaciones sísmicas usando datos de InSAR y otras fuentes.

### Características:
- ✅ Entrenamiento interactivo con visualización en tiempo real
- ✅ Soporte para clasificación y regresión
- ✅ Carga segmentada de datasets grandes
- ✅ Métricas de evaluación completas
- ✅ Guardado automático de modelos entrenados

## 1. 📚 Importación de Librerías y Configuración

Importamos todas las librerías necesarias para el entrenamiento de modelos sísmicos.

In [ ]:
# Importación de librerías necesarias
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import pandas as pd
import h5py
import json
from typing import Dict, List, Tuple, Optional
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Configuración del entorno
plt.style.use('default')
sns.set_palette("husl")

# Añadir el directorio backend al path
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
backend_dir = os.path.join(notebook_dir, 'backend')
if backend_dir not in sys.path:
    sys.path.insert(0, backend_dir)

print("✅ Librerías importadas correctamente")
print(f"📁 Directorio backend: {backend_dir}")
print(f"🖥️  PyTorch versión: {torch.__version__}")
print(f"🎯 CUDA disponible: {torch.cuda.is_available()}")

## 2. 📊 Carga y Exploración de Datos Sísmicos

Exploramos los datasets disponibles y configuramos la carga de datos con chunking para manejar datasets grandes.

In [ ]:
# Exploración de datasets disponibles
def explore_datasets():
    """Explora los datasets disponibles en el directorio backend/datasets"""
    datasets_dir = os.path.join(backend_dir, 'datasets')

    if not os.path.exists(datasets_dir):
        print(f"❌ Directorio de datasets no encontrado: {datasets_dir}")
        return []

    print(f"📂 Explorando datasets en: {datasets_dir}")
    datasets = []

    for file in os.listdir(datasets_dir):
        if file.endswith('.h5'):
            filepath = os.path.join(datasets_dir, file)
            try:
                with h5py.File(filepath, 'r') as f:
                    info = {
                        'filename': file,
                        'filepath': filepath,
                        'size_mb': os.path.getsize(filepath) / (1024 * 1024),
                        'groups': list(f.keys())
                    }

                    # Obtener información detallada del dataset
                    if len(f.keys()) > 0:
                        main_group = list(f.keys())[0]
                        grupo = f[main_group]
                        if 'secuencias' in grupo:
                            shape = grupo['secuencias'].shape
                            info['shape'] = shape
                            info['num_samples'] = shape[0]
                            info['seq_length'] = shape[1]
                            info['grid_height'] = shape[2]
                            info['grid_width'] = shape[3]

                        if 'etiquetas' in grupo:
                            info['has_labels'] = True
                            info['task_type'] = 'classification'
                        elif 'regresion' in f:
                            info['has_labels'] = True
                            info['task_type'] = 'regression'
                        else:
                            info['has_labels'] = False

                    datasets.append(info)

            except Exception as e:
                print(f"⚠️  Error al leer {file}: {e}")

    return datasets

# Ejecutar exploración
datasets_info = explore_datasets()

# Mostrar información de datasets
if datasets_info:
    print(f"\n📊 Encontrados {len(datasets_info)} datasets:")
    for i, ds in enumerate(datasets_info, 1):
        print(f"\n{i}. {ds['filename']}")
        print(f"   📏 Tamaño: {ds['size_mb']:.1f} MB")
        if 'shape' in ds:
            print(f"   📐 Forma: {ds['shape']} (muestras × tiempo × altura × ancho)")
        if 'task_type' in ds:
            print(f"   🎯 Tipo: {ds['task_type']}")
        print(f"   📂 Grupos: {ds['groups']}")
else:
    print("❌ No se encontraron datasets HDF5")
    print("💡 Ejecuta 'python main.py generate' en el directorio backend para crear datasets sintéticos")

In [ ]:
# Función para generar datasets automáticamente si no existen
def generate_datasets_if_needed():
    """Genera datasets sintéticos automáticamente si no existen"""
    datasets_dir = os.path.join(backend_dir, 'datasets')

    # Verificar si ya existen datasets
    if os.path.exists(datasets_dir):
        existing_files = [f for f in os.listdir(datasets_dir) if f.endswith('.h5')]
        if existing_files:
            print(f"✅ Ya existen {len(existing_files)} datasets: {existing_files}")
            return True

    print("📊 No se encontraron datasets. Generando datos sintéticos automáticamente...")
    print("🔧 Esto puede tomar varios minutos...")

    try:
        # Importar y ejecutar la generación de datasets
        import subprocess
        import sys

        # Ejecutar el comando de generación desde el directorio backend
        result = subprocess.run([
            sys.executable, 'main.py', 'generate'
        ], cwd=backend_dir, capture_output=True, text=True)

        if result.returncode == 0:
            print("✅ Datasets generados exitosamente!")
            print(result.stdout)
            return True
        else:
            print("❌ Error al generar datasets:")
            print(result.stderr)
            return False

    except Exception as e:
        print(f"❌ Error ejecutando generación de datasets: {e}")
        return False

# Generar datasets automáticamente si es necesario
datasets_available = generate_datasets_if_needed()

if datasets_available:
    # Re-explorar datasets después de generarlos
    datasets_info = explore_datasets()
else:
    print("❌ No se pudieron generar los datasets. Verifica la configuración del backend.")

## 3. ⚙️ Configuración del Entrenamiento

Configura los parámetros del entrenamiento usando controles interactivos.

In [ ]:
# Widgets interactivos para configuración del entrenamiento
if datasets_info:
    # Selector de dataset
    dataset_selector = widgets.Dropdown(
        options=[(f"{ds['filename']} ({ds['task_type']})", ds['filepath']) for ds in datasets_info],
        description='Dataset:',
        style={'description_width': 'initial'}
    )

    # Selector de tarea (si no se puede inferir del dataset)
    task_selector = widgets.Dropdown(
        options=['classification', 'regression'],
        value='classification',
        description='Tarea:',
        style={'description_width': 'initial'}
    )

    # Parámetros de entrenamiento
    epochs_slider = widgets.IntSlider(
        value=30, min=5, max=200, step=5,
        description='Épocas:',
        style={'description_width': 'initial'}
    )

    batch_size_slider = widgets.IntSlider(
        value=4, min=1, max=32, step=1,
        description='Batch Size:',
        style={'description_width': 'initial'}
    )

    learning_rate_slider = widgets.FloatLogSlider(
        value=1e-4, base=10, min=-6, max=-2, step=0.5,
        description='Learning Rate:',
        style={'description_width': 'initial'}
    )

    chunk_size_slider = widgets.IntSlider(
        value=1000, min=100, max=5000, step=100,
        description='Chunk Size:',
        style={'description_width': 'initial'}
    )

    # Botón de entrenamiento
    train_button = widgets.Button(
        description='🚀 Iniciar Entrenamiento',
        button_style='success',
        tooltip='Comenzar el entrenamiento del modelo'
    )

    # Área de salida
    output_area = widgets.Output()

    # Mostrar widgets
    display(widgets.VBox([
        widgets.HTML("<h4>📊 Configuración del Dataset</h4>"),
        dataset_selector,
        task_selector,
        widgets.HTML("<h4>🔧 Parámetros de Entrenamiento</h4>"),
        epochs_slider,
        batch_size_slider,
        learning_rate_slider,
        chunk_size_slider,
        widgets.HTML("<h4>🎯 Control del Entrenamiento</h4>"),
        train_button,
        output_area
    ]))

    # Función para determinar la tarea automáticamente
    def update_task_selector(change):
        if change['type'] == 'change' and change['name'] == 'value':
            selected_path = change['new']
            for ds in datasets_info:
                if ds['filepath'] == selected_path:
                    if 'task_type' in ds:
                        task_selector.value = ds['task_type']
                    break

    dataset_selector.observe(update_task_selector)

else:
    print("❌ No hay datasets disponibles. Genera datasets primero ejecutando:")
    print("   cd backend && python main.py generate")

In [ ]:
# Función de entrenamiento automático con parámetros por defecto
def train_model_auto():
    """Entrenamiento automático con parámetros optimizados"""
    if not datasets_info:
        print("❌ No hay datasets disponibles para entrenar")
        return

    # Usar el primer dataset disponible (clasificación por defecto)
    selected_dataset = None
    for ds in datasets_info:
        if 'classification' in ds.get('task_type', ''):
            selected_dataset = ds
            break

    if not selected_dataset:
        selected_dataset = datasets_info[0]  # Usar cualquier dataset disponible

    print("🚀 Iniciando entrenamiento automático..."    print(f"📊 Dataset seleccionado: {selected_dataset['filename']}")
    print(f"🎯 Tarea: {selected_dataset.get('task_type', 'classification')}")
    print("-" * 50)

    # Parámetros optimizados por defecto
    config = {
        'dataset_path': selected_dataset['filepath'],
        'task_type': selected_dataset.get('task_type', 'classification'),
        'epochs': 50,
        'batch_size': 8,
        'learning_rate': 1e-4,
        'chunk_size': 1000
    }

    try:
        # Crear dataset
        dataset = DeformationDataset(
            config['dataset_path'],
            task_type=config['task_type'],
            chunk_size=config['chunk_size']
        )

        # Dividir en train/val
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

        # Crear dataloaders
        train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

        # Crear modelo
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = create_model(
            d_model=256,
            nhead=8,
            num_encoder_layers=6,
            seq_length=dataset.seq_length,
            grid_size=(dataset.grid_size[0], dataset.grid_size[1]),
            num_classes=3 if config['task_type'] == 'classification' else 1,
            task_type=config['task_type']
        ).to(device)

        print(f"📊 Modelo creado: {sum(p.numel() for p in model.parameters())} parámetros")
        print(f"🖥️  Dispositivo: {device}")

        # Configurar optimizador y loss
        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
        criterion = nn.CrossEntropyLoss() if config['task_type'] == 'classification' else nn.MSELoss()

        # Entrenamiento
        train_losses = []
        val_losses = []
        train_accuracies = [] if config['task_type'] == 'classification' else []
        val_accuracies = [] if config['task_type'] == 'classification' else []

        best_val_loss = float('inf')
        patience = 10
        patience_counter = 0

        print("🏃 Iniciando entrenamiento automático...")
        print("-" * 50)

        for epoch in range(config['epochs']):
            # Entrenamiento
            model.train()
            epoch_train_loss = 0
            epoch_train_correct = 0
            epoch_train_total = 0

            for batch_x, batch_y in train_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)

                optimizer.zero_grad()
                outputs = model(batch_x)

                if config['task_type'] == 'classification':
                    loss = criterion(outputs, batch_y)
                    _, predicted = torch.max(outputs.data, 1)
                    epoch_train_correct += (predicted == batch_y).sum().item()
                    epoch_train_total += batch_y.size(0)
                else:
                    loss = criterion(outputs.squeeze(), batch_y)

                loss.backward()
                optimizer.step()
                epoch_train_loss += loss.item()

            avg_train_loss = epoch_train_loss / len(train_loader)
            train_accuracy = epoch_train_correct / epoch_train_total if epoch_train_total > 0 else 0

            # Validación
            model.eval()
            epoch_val_loss = 0
            epoch_val_correct = 0
            epoch_val_total = 0

            with torch.no_grad():
                for batch_x, batch_y in val_loader:
                    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

                    outputs = model(batch_x)

                    if config['task_type'] == 'classification':
                        loss = criterion(outputs, batch_y)
                        _, predicted = torch.max(outputs.data, 1)
                        epoch_val_correct += (predicted == batch_y).sum().item()
                        epoch_val_total += batch_y.size(0)
                    else:
                        loss = criterion(outputs.squeeze(), batch_y)

                    epoch_val_loss += loss.item()

            avg_val_loss = epoch_val_loss / len(val_loader)
            val_accuracy = epoch_val_correct / epoch_val_total if epoch_val_total > 0 else 0

            # Guardar métricas
            train_losses.append(avg_train_loss)
            val_losses.append(avg_val_loss)
            if config['task_type'] == 'classification':
                train_accuracies.append(train_accuracy)
                val_accuracies.append(val_accuracy)

            # Mostrar progreso cada 5 épocas
            if (epoch + 1) % 5 == 0 or epoch == 0:
                print(f"Época {epoch+1:3d}: Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
                if config['task_type'] == 'classification':
                    print(f"            Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}")

            # Early stopping
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                patience_counter = 0
                # Guardar mejor modelo
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                checkpoint_path = f"checkpoints_{config['task_type']}_auto_{timestamp}_best.pth"
                save_model_checkpoint(model, optimizer, epoch, best_val_loss, checkpoint_path)
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"⏹️  Early stopping en época {epoch+1}")
                    break

        print("✅ Entrenamiento automático completado!")

        # Visualizar resultados
        plot_training_results(train_losses, val_losses, train_accuracies, val_accuracies, config['task_type'])

        # Mostrar resumen final
        print("
📊 Resumen del Entrenamiento Automático:"        print(f"   Modelo guardado: {checkpoint_path}")
        print(f"   Épocas completadas: {len(train_losses)}")
        print(f"   Mejor pérdida de validación: {best_val_loss:.4f}")
        if config['task_type'] == 'classification' and val_accuracies:
            print(f"   Mejor precisión de validación: {max(val_accuracies):.4f}")

        return checkpoint_path

    except Exception as e:
        print(f"❌ Error en entrenamiento automático: {e}")
        import traceback
        traceback.print_exc()
        return None

# Botón de entrenamiento automático
auto_train_button = widgets.Button(
    description='🤖 Entrenamiento Automático',
    button_style='success',
    tooltip='Entrenar automáticamente con parámetros optimizados'
)

auto_output_area = widgets.Output()

def auto_train_click(b):
    with auto_output_area:
        clear_output(wait=True)
        train_model_auto()

auto_train_button.on_click(auto_train_click)

# Mostrar botón de entrenamiento automático
if datasets_info:
    display(widgets.VBox([
        widgets.HTML("<h4>🚀 Entrenamiento Automático</h4>"),
        widgets.HTML("Entrenamiento completo con parámetros optimizados por defecto"),
        auto_train_button,
        auto_output_area
    ]))

## 4. 🚀 Entrenamiento del Modelo

Ejecuta el entrenamiento con visualización en tiempo real del progreso.

In [ ]:
# Importar funciones del backend
try:
    from model_architecture import create_model, save_model_checkpoint, load_model_checkpoint
    from model_training.train_model import DeformationDataset
    print("✅ Funciones del backend importadas correctamente")
except ImportError as e:
    print(f"❌ Error al importar funciones del backend: {e}")
    print("💡 Asegúrate de que el directorio backend esté en el path")

# Función de entrenamiento interactivo
def train_model_interactive(b):
    """Función que maneja el entrenamiento cuando se presiona el botón"""
    with output_area:
        clear_output(wait=True)

        try:
            # Obtener configuración
            dataset_path = dataset_selector.value
            task_type = task_selector.value
            epochs = epochs_slider.value
            batch_size = batch_size_slider.value
            learning_rate = learning_rate_slider.value
            chunk_size = chunk_size_slider.value

            print("🚀 Iniciando entrenamiento interactivo..."            print(f"📊 Dataset: {os.path.basename(dataset_path)}")
            print(f"🎯 Tarea: {task_type}")
            print(f"📈 Épocas: {epochs}")
            print(f"📦 Batch size: {batch_size}")
            print(f"🎓 Learning rate: {learning_rate}")
            print(f"🧩 Chunk size: {chunk_size}")
            print("-" * 50)

            # Crear dataset
            print("📚 Creando dataset...")
            dataset = DeformationDataset(dataset_path, task_type=task_type, chunk_size=chunk_size)

            # Dividir en train/val
            train_size = int(0.8 * len(dataset))
            val_size = len(dataset) - train_size
            train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

            # Crear dataloaders
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

            print(f"✅ Dataset creado: {len(train_dataset)} train, {len(val_dataset)} val")

            # Crear modelo
            print("🏗️  Creando modelo...")
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            model = create_model(
                d_model=256,
                nhead=8,
                num_encoder_layers=6,
                seq_length=dataset.seq_length,
                grid_size=(dataset.grid_size[0], dataset.grid_size[1]),
                num_classes=3 if task_type == 'classification' else 1,
                task_type=task_type
            ).to(device)

            print(f"📊 Modelo creado con {sum(p.numel() for p in model.parameters())} parámetros")
            print(f"🖥️  Dispositivo: {device}")

            # Configurar optimizador y loss
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)
            if task_type == 'classification':
                criterion = nn.CrossEntropyLoss()
            else:
                criterion = nn.MSELoss()

            # Entrenamiento
            print("🏃 Iniciando entrenamiento...")
            print("-" * 50)

            # Listas para métricas
            train_losses = []
            val_losses = []
            train_accuracies = [] if task_type == 'classification' else []
            val_accuracies = [] if task_type == 'classification' else []

            best_val_loss = float('inf')
            patience = 10
            patience_counter = 0

            for epoch in range(epochs):
                # Entrenamiento
                model.train()
                epoch_train_loss = 0
                epoch_train_correct = 0
                epoch_train_total = 0

                train_pbar = tqdm(train_loader, desc=f'Época {epoch+1}/{epochs} [Train]')
                for batch_x, batch_y in train_pbar:
                    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

                    optimizer.zero_grad()
                    outputs = model(batch_x)

                    if task_type == 'classification':
                        loss = criterion(outputs, batch_y)
                        _, predicted = torch.max(outputs.data, 1)
                        epoch_train_correct += (predicted == batch_y).sum().item()
                        epoch_train_total += batch_y.size(0)
                    else:
                        loss = criterion(outputs.squeeze(), batch_y)
                        # Para regresión, calculamos una "precisión" basada en error relativo
                        pred_flat = outputs.squeeze().view(-1)
                        true_flat = batch_y.view(-1)
                        accuracy = (torch.abs(pred_flat - true_flat) / (torch.abs(true_flat) + 1e-8) < 0.1).float().mean().item()
                        epoch_train_correct += accuracy * batch_y.numel()
                        epoch_train_total += batch_y.numel()

                    loss.backward()
                    optimizer.step()

                    epoch_train_loss += loss.item()
                    train_pbar.set_postfix({'loss': f'{loss.item():.4f}'})

                avg_train_loss = epoch_train_loss / len(train_loader)
                train_accuracy = epoch_train_correct / epoch_train_total if epoch_train_total > 0 else 0

                # Validación
                model.eval()
                epoch_val_loss = 0
                epoch_val_correct = 0
                epoch_val_total = 0

                with torch.no_grad():
                    val_pbar = tqdm(val_loader, desc=f'Época {epoch+1}/{epochs} [Val]')
                    for batch_x, batch_y in val_pbar:
                        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

                        outputs = model(batch_x)

                        if task_type == 'classification':
                            loss = criterion(outputs, batch_y)
                            _, predicted = torch.max(outputs.data, 1)
                            epoch_val_correct += (predicted == batch_y).sum().item()
                            epoch_val_total += batch_y.size(0)
                        else:
                            loss = criterion(outputs.squeeze(), batch_y)
                            pred_flat = outputs.squeeze().view(-1)
                            true_flat = batch_y.view(-1)
                            accuracy = (torch.abs(pred_flat - true_flat) / (torch.abs(true_flat) + 1e-8) < 0.1).float().mean().item()
                            epoch_val_correct += accuracy * batch_y.numel()
                            epoch_val_total += batch_y.numel()

                        epoch_val_loss += loss.item()
                        val_pbar.set_postfix({'loss': f'{loss.item():.4f}'})

                avg_val_loss = epoch_val_loss / len(val_loader)
                val_accuracy = epoch_val_correct / epoch_val_total if epoch_val_total > 0 else 0

                # Guardar métricas
                train_losses.append(avg_train_loss)
                val_losses.append(avg_val_loss)
                if task_type == 'classification':
                    train_accuracies.append(train_accuracy)
                    val_accuracies.append(val_accuracy)

                # Mostrar progreso
                print(f"Época {epoch+1:3d}: Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
                if task_type == 'classification':
                    print(f"            Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}")
                else:
                    print(f"            Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}")

                # Early stopping
                if avg_val_loss < best_val_loss:
                    best_val_loss = avg_val_loss
                    patience_counter = 0
                    # Guardar mejor modelo
                    checkpoint_path = f"checkpoints_{task_type}_{os.path.basename(dataset_path).split('.')[0]}_best.pth"
                    save_model_checkpoint(model, optimizer, epoch, best_val_loss, checkpoint_path)
                    print(f"💾 Mejor modelo guardado: {checkpoint_path}")
                else:
                    patience_counter += 1
                    if patience_counter >= patience:
                        print(f"⏹️  Early stopping en época {epoch+1}")
                        break

            print("✅ Entrenamiento completado!")

            # Visualizar resultados
            plot_training_results(train_losses, val_losses, train_accuracies, val_accuracies, task_type)

        except Exception as e:
            print(f"❌ Error durante el entrenamiento: {e}")
            import traceback
            traceback.print_exc()

# Conectar el botón a la función
if 'train_button' in locals():
    train_button.on_click(train_model_interactive)

## 5. 📈 Visualización de Resultados

Visualiza las métricas de entrenamiento y evalúa el rendimiento del modelo.

In [ ]:
# Función para visualizar resultados del entrenamiento
def plot_training_results(train_losses, val_losses, train_accuracies, val_accuracies, task_type):
    """Visualiza las curvas de entrenamiento"""
    fig = make_subplots(
        rows=1, cols=2 if task_type == 'classification' else 1,
        subplot_titles=['Pérdida (Loss)', 'Precisión (Accuracy)' if task_type == 'classification' else None],
        specs=[[{"secondary_y": False}, {"secondary_y": False}] if task_type == 'classification' else [{"secondary_y": False}]]
    )

    epochs = list(range(1, len(train_losses) + 1))

    # Gráfico de pérdida
    fig.add_trace(
        go.Scatter(x=epochs, y=train_losses, mode='lines+markers', name='Train Loss',
                  line=dict(color='blue', width=2)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=epochs, y=val_losses, mode='lines+markers', name='Val Loss',
                  line=dict(color='red', width=2)),
        row=1, col=1
    )

    # Gráfico de precisión (solo para clasificación)
    if task_type == 'classification' and train_accuracies and val_accuracies:
        fig.add_trace(
            go.Scatter(x=epochs, y=train_accuracies, mode='lines+markers', name='Train Accuracy',
                      line=dict(color='green', width=2)),
            row=1, col=2
        )
        fig.add_trace(
            go.Scatter(x=epochs, y=val_accuracies, mode='lines+markers', name='Val Accuracy',
                      line=dict(color='orange', width=2)),
            row=1, col=2
        )

    # Configurar layout
    fig.update_layout(
        title_text="📊 Resultados del Entrenamiento",
        title_x=0.5,
        height=500,
        showlegend=True
    )

    fig.update_xaxes(title_text="Época")
    fig.update_yaxes(title_text="Loss", row=1, col=1)
    if task_type == 'classification':
        fig.update_yaxes(title_text="Accuracy", row=1, col=2)

    fig.show()

    # Mostrar métricas finales
    print("
📊 Métricas Finales:"    print(f"   Pérdida de entrenamiento final: {train_losses[-1]:.4f}")
    print(f"   Pérdida de validación final: {val_losses[-1]:.4f}")
    if task_type == 'classification' and train_accuracies and val_accuracies:
        print(f"   Precisión de entrenamiento final: {train_accuracies[-1]:.4f}")
        print(f"   Precisión de validación final: {val_accuracies[-1]:.4f}")

    # Encontrar mejores métricas
    best_epoch = val_losses.index(min(val_losses)) + 1
    print(f"   Mejor época: {best_epoch}")
    print(f"   Mejor pérdida de validación: {min(val_losses):.4f}")

# Función para evaluar el modelo en detalle
def evaluate_model_detailed(model_path, dataset_path, task_type):
    """Evaluación detallada del modelo"""
    try:
        print(f"🔍 Evaluando modelo: {model_path}")

        # Cargar modelo
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        checkpoint = load_model_checkpoint(model_path, device)

        # Crear dataset de prueba
        dataset = DeformationDataset(dataset_path, task_type=task_type, chunk_size=1000)

        # Usar el 20% final para prueba
        test_size = int(0.2 * len(dataset))
        train_val_size = len(dataset) - test_size
        _, test_dataset = random_split(dataset, [train_val_size, test_size])

        test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

        # Evaluar
        model = checkpoint['model']
        model.eval()

        all_predictions = []
        all_labels = []

        with torch.no_grad():
            for batch_x, batch_y in tqdm(test_loader, desc="Evaluando"):
                batch_x = batch_x.to(device)
                outputs = model(batch_x)

                if task_type == 'classification':
                    _, predicted = torch.max(outputs.data, 1)
                    all_predictions.extend(predicted.cpu().numpy())
                    all_labels.extend(batch_y.numpy())
                else:
                    all_predictions.extend(outputs.squeeze().cpu().numpy())
                    all_labels.extend(batch_y.numpy())

        # Calcular métricas
        if task_type == 'classification':
            from sklearn.metrics import classification_report, confusion_matrix
            print("\n📊 Reporte de Clasificación:")
            print(classification_report(all_labels, all_predictions))

            # Matriz de confusión
            cm = confusion_matrix(all_labels, all_predictions)
            plt.figure(figsize=(8, 6))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                       xticklabels=['Precursor', 'Normal', 'Post-terremoto'],
                       yticklabels=['Precursor', 'Normal', 'Post-terremoto'])
            plt.title('Matriz de Confusión')
            plt.ylabel('Verdadero')
            plt.xlabel('Predicho')
            plt.show()
        else:
            from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
            mse = mean_squared_error(all_labels, all_predictions)
            mae = mean_absolute_error(all_labels, all_predictions)
            r2 = r2_score(all_labels, all_predictions)

            print("
📊 Métricas de Regresión:"            print(f"   MSE: {mse:.4f}")
            print(f"   MAE: {mae:.4f}")
            print(f"   R²: {r2:.4f}")

            # Gráfico de predicción vs real
            plt.figure(figsize=(10, 6))
            plt.scatter(all_labels, all_predictions, alpha=0.5)
            plt.plot([min(all_labels), max(all_labels)], [min(all_labels), max(all_labels)], 'r--')
            plt.xlabel('Valores Reales')
            plt.ylabel('Predicciones')
            plt.title('Predicción vs Real')
            plt.grid(True)
            plt.show()

    except Exception as e:
        print(f"❌ Error en evaluación detallada: {e}")

print("✅ Funciones de visualización y evaluación cargadas")

## 6. 📝 Instrucciones de Uso

### 🚀 Inicio Rápido:
1. **Ejecuta todas las celdas** en orden (Cell → Run All)
2. **Configura los parámetros** usando los controles interactivos
3. **Presiona "🚀 Iniciar Entrenamiento"** para comenzar
4. **Monitorea el progreso** en tiempo real
5. **Revisa los resultados** en las visualizaciones

### 📊 Datasets Disponibles:
- **falla_anatolia**: Datos de la falla de Anatolia (Turquía)
- **cinturon_fuego_pacifico**: Datos del cinturón de fuego del Pacífico

### 🎯 Tipos de Tarea:
- **classification**: Predice si es precursor, normal, o post-terremoto
- **regression**: Predice valores continuos de deformación

### ⚙️ Parámetros Recomendados:
- **Épocas**: 30-100 (dependiendo del dataset)
- **Batch Size**: 4-16 (ajusta según memoria disponible)
- **Learning Rate**: 1e-4 a 1e-3
- **Chunk Size**: 1000-2000 (para carga eficiente)

### 💾 Modelos Guardados:
Los modelos se guardan automáticamente en el directorio `backend/checkpoints_*` con el mejor rendimiento en validación.

### 🔧 Solución de Problemas:
- Si no hay datasets: ejecuta `cd backend && python main.py generate`
- Si hay errores de memoria: reduce batch_size o chunk_size
- Si el entrenamiento es lento: verifica que CUDA esté disponible

---

## 🎉 ¡Listo para Entrenar!

Este notebook proporciona una interfaz completa y interactiva para entrenar modelos de predicción sísmica. ¡Experimenta con diferentes configuraciones y datasets!

## 7. ⚡ Modo Automático Completo

Ejecuta todo el proceso automáticamente: generación de datos + entrenamiento.

In [ ]:
# Función para ejecutar todo el proceso automáticamente
def run_complete_auto_pipeline():
    """Ejecuta el pipeline completo: generación de datos + entrenamiento automático"""
    print("🚀 Iniciando Pipeline Automático Completo")
    print("=" * 60)

    # Paso 1: Generar datasets si no existen
    print("📊 PASO 1: Generación de Datasets")
    datasets_ok = generate_datasets_if_needed()

    if not datasets_ok:
        print("❌ Error en generación de datasets. Abortando pipeline.")
        return False

    # Re-explorar datasets
    global datasets_info
    datasets_info = explore_datasets()

    if not datasets_info:
        print("❌ No se encontraron datasets después de la generación. Abortando pipeline.")
        return False

    print("\n" + "=" * 60)
    print("🤖 PASO 2: Entrenamiento Automático")
    # Paso 2: Entrenamiento automático
    model_path = train_model_auto()

    if model_path:
        print("\n" + "=" * 60)
        print("✅ PIPELINE COMPLETADO EXITOSAMENTE!")
        print(f"📁 Modelo guardado en: {model_path}")
        print("\n🎯 El modelo está listo para usar en inferencia!")
        print("💡 Puedes usar el modelo con: python main.py seismic")
        return True
    else:
        print("❌ Error en el entrenamiento. Pipeline incompleto.")
        return False

# Botón para pipeline completo automático
complete_auto_button = widgets.Button(
    description='🚀 Pipeline Completo Automático',
    button_style='primary',
    tooltip='Generar datos + entrenar automáticamente (todo en uno)',
    layout=widgets.Layout(width='300px', height='50px')
)

complete_output_area = widgets.Output()

def complete_auto_click(b):
    with complete_output_area:
        clear_output(wait=True)
        run_complete_auto_pipeline()

complete_auto_button.on_click(complete_auto_click)

# Mostrar botón de pipeline completo
display(widgets.VBox([
    widgets.HTML("<h3>⚡ Pipeline Automático Completo</h3>"),
    widgets.HTML("🔧 Genera datasets sintéticos automáticamente<br>🤖 Entrena modelo con parámetros optimizados<br>📊 Visualiza resultados completos"),
    widgets.HTML("<br><strong>¡Solo presiona el botón y espera!</strong>"),
    complete_auto_button,
    complete_output_area
]))

print("🎯 El notebook está listo para usar en modo automático o interactivo!")

---

## 📊 Estado del Sistema

Ejecuta esta celda para verificar el estado actual del sistema:

In [ ]:
# Verificación del estado del sistema
def check_system_status():
    """Verifica el estado completo del sistema de entrenamiento sísmico"""
    print("🔍 Verificando estado del sistema...")
    print("=" * 50)

    # Verificar backend
    backend_ok = os.path.exists(backend_dir)
    print(f"📁 Backend: {'✅' if backend_ok else '❌'} ({backend_dir})")

    # Verificar datasets
    datasets_dir = os.path.join(backend_dir, 'datasets')
    datasets_ok = os.path.exists(datasets_dir)
    dataset_files = []
    if datasets_ok:
        dataset_files = [f for f in os.listdir(datasets_dir) if f.endswith('.h5')]
    print(f"📊 Datasets: {'✅' if datasets_ok and dataset_files else '❌'} ({len(dataset_files)} archivos)")

    # Verificar modelos
    models_dir = os.path.join(backend_dir, 'models')
    models_ok = os.path.exists(models_dir)
    print(f"🤖 Modelos: {'✅' if models_ok else '❌'} ({models_dir})")

    # Verificar checkpoints
    checkpoints_dir = os.path.join(backend_dir, 'checkpoints')
    checkpoints_ok = os.path.exists(checkpoints_dir)
    checkpoint_files = []
    if checkpoints_ok:
        checkpoint_files = [f for f in os.listdir(checkpoints_dir) if f.endswith('.pth')]
    print(f"💾 Checkpoints: {'✅' if checkpoints_ok else '❌'} ({len(checkpoint_files)} modelos)")

    # Verificar PyTorch y CUDA
    cuda_available = torch.cuda.is_available()
    device_count = torch.cuda.device_count() if cuda_available else 0
    print(f"🖥️  PyTorch: ✅ (CUDA: {'✅' if cuda_available else '❌'}, GPUs: {device_count})")

    # Verificar imports críticos
    imports_ok = True
    try:
        from model_architecture import create_model
        from model_training.train_model import DeformationDataset
        print("📚 Imports: ✅ (model_architecture, train_model)")
    except ImportError as e:
        print(f"📚 Imports: ❌ ({e})")
        imports_ok = False

    # Resumen
    all_ok = backend_ok and imports_ok
    datasets_ready = datasets_ok and dataset_files

    print("\n" + "=" * 50)
    print("📋 RESUMEN DEL SISTEMA:")
    print(f"   Backend configurado: {'✅' if all_ok else '❌'}")
    print(f"   Datasets listos: {'✅' if datasets_ready else '❌'}")
    print(f"   Entrenamiento posible: {'✅' if all_ok and (datasets_ready or True) else '❌'}")

    if all_ok and not datasets_ready:
        print("   💡 Los datasets se generarán automáticamente en modo automático")
    elif all_ok and datasets_ready:
        print("   🎯 Sistema completamente listo para entrenamiento!")

    return all_ok

# Ejecutar verificación
system_ready = check_system_status()

if system_ready:
    print("\n🎉 ¡Sistema listo! Elige tu modo de entrenamiento:")
    print("   🚀 Pipeline Completo Automático (recomendado)")
    print("   🎛️  Modo Interactivo (control total)")
    print("   🤖 Entrenamiento Automático (parámetros optimizados)")
else:
    print("\n⚠️  Sistema necesita configuración. Verifica los errores arriba.")